In [4]:
import json
import math
import affine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr

In [6]:
folder='//nau.froot.nau.edu/cirrus/scratch/lkp58/datasets/AVIRIS/sites/BONA/ang20180723t193341_rfl_v2r2/'
# print metadata keys from header file
with open(folder + "ang20180723t193341_corr_v2r2_img.hdr","r") as f:
    print("Metadata keys:\n"+", ".join(
        [ln.strip().split(" = ")[0] for ln in f.readlines() if " = " in ln]))

Metadata keys:
description, samples, lines, bands, header offset, file type, data type, interleave, byte order, map info, crosstrack scatter file, wavelength units, flat field file, spectral scatter file, correction factors, wavelength file, wavelength, radiance version, fwhm, bbl, rcc file, smoothing factors, data ignore value, bad pixel map


In [7]:
# open the ENVI file
img = gdal.Open(folder + "ang20180723t193341_corr_v2r2_img")

nbands = img.RasterCount
nrows = img.RasterYSize
ncols = img.RasterXSize

print("\n".join(["Bands:\t"+str(nbands),"Rows:\t"+str(nrows),"Cols:\t"+str(ncols)]))

Bands:	425
Rows:	3527
Cols:	860


In [8]:
# get band information
# band descriptions. we reference this dictionary throughout the tutorial.
band_dictionary = {
    "visible-violet": {'lower': 375, 'upper': 450, 'color': 'violet'},
    "visible-blue": {'lower': 450, 'upper': 485, 'color': 'blue'},
    "visible-cyan": {'lower': 485, 'upper': 500, 'color': 'cyan'},
    "visible-green": {'lower': 500, 'upper': 565, 'color': 'green'},
    "visible-yellow": {'lower': 565, 'upper': 590, 'color': 'yellow'},
    "visible-orange": {'lower': 590, 'upper': 625, 'color': 'orange'},
    "visible-red": {'lower': 625, 'upper': 740, 'color': 'red'},
    "near-infrared": {'lower': 740, 'upper': 1100, 'color': 'gray'},
    "shortwave-infrared": {'lower': 1100, 'upper': 2500, 'color': 'white'}
}

# function to classify bands
between = lambda wavelength, region: region['lower'] < wavelength <= region['upper']
def classifier(band):
    for region, limits in band_dictionary.items():
        if between(band, limits):
            return(region)

# lists of band numbers, band centers, and em classes
band_numbers = [int(b.split("_")[1]) for b in img.GetMetadata().keys() if b != "wavelength_units"]
band_centers = [float(b.split(" ")[0]) for b in img.GetMetadata().values() if b != "Nanometers"]
em_regions = [classifier(b) for b in band_centers]

# data frame describing bands
bands = pd.DataFrame({ 
    "Band number": band_numbers, 
    "Band center (nm)": band_centers, 
    "EM region": em_regions }, index = band_numbers).sort_index()

# print the first ten rows
bands.head(10)

,Band number,Band center (nm),EM region
1,1,376.86,visible-violet
2,2,381.87,visible-violet
3,3,386.88,visible-violet
4,4,391.89,visible-violet
5,5,396.89,visible-violet
6,6,401.90,visible-violet
7,7,406.91,visible-violet
8,8,411.92,visible-violet
9,9,416.93,visible-violet
10,10,421.94,visible-violet


In [13]:
sites = "E:/research/datasets/NEON/BONA/veg_shp_UTM_6N/NEON_BONA_veg_UTM_6N.shp"

# open with ogr
driver = ogr.GetDriverByName("ESRI Shapefile")
tree_sites = driver.Open(sites, 0)

# get the first feature in the shapefile as JSON
site1 = json.loads(tree_sites[0].GetFeature(0).ExportToJson())
site1

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [476359.03159147024, 7226226.870952932]},
 'properties': {'indvdID': 'NEON.PLA.D19.BONA.00156',
  'nmdLctn': 'BONA_079.basePlot.vst',
  'domanID': 'D19',
  'siteID': 'BONA',
  'plotID': 'BONA_079',
  'uid_x': '9ee1716a-e1e3-40ef-a696-3661a69dc77a',
  'date_x': '2016-06-15',
  'evntID_x': 'vst_BONA_2016',
  'sbpltID_x': None,
  'tmpStID': None,
  'tagStts': None,
  'grwthFr': 'single bole tree',
  'plntStt': 'Standing dead',
  'stmDmtr': 21.0,
  'msrmntH': 140.0,
  'height': None,
  'bsCrwnH': None,
  'brkHght': None,
  'brkDmtr': None,
  'mxCrwnD': None,
  'nntyCrD': None,
  'cnpyPst': None,
  'shape': None,
  'bslStmD': None,
  'bslSDMH': None,
  'mxBsCrD': None,
  'nntyBCD': None,
  'dndrmID': None,
  'intlGMD': None,
  'intlBSD': None,
  'intlDnG': None,
  'dndrmtH': None,
  'dndrmtG': None,
  'dndrmtC': None,
  'bndStmD': None,
  'rmrks_x': None,
  'rcrddBy_x': 'kjorgenson@field-ops.org',
  'msrdBy_x': 'mgrandfield

In [26]:
print("ENVI image WKT: \n"+str(img.GetProjectionRef()))
print("\nShapefile WKT: \n"+str(tree_sites[0].GetSpatialRef()))

ENVI image WKT: 
PROJCS["UTM Zone 6, Northern Hemisphere",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-147],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]

Shapefile WKT: 
PROJCS["WGS_1984_UTM_Zone_6N",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_84",6378137.0,298.257223563]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["False_Easting",500000.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",-147.0],
    PARAMETER["Scale_Factor",0.9996],
    PARAMETER["Latitude_Of_Origin",0

In [37]:
# shapefiles have a nested structure: layer(s) -> feature(s) -> geometry
lyr = tree_sites.GetLayer() # get the only layer in the shapefile
feat = lyr.GetFeature(0)          # get the first feature in the layer (1 feature per site)
geom = feat.GetGeometryRef()      # get the feature's geometry


# get transform for decimal degrees
from_srs = lyr.GetSpatialRef()                                         # get shapefile srs def
to_srs = osr.SpatialReference()                                        # init ogr srs object
to_srs.ImportFromEPSG(4326)                                            # import wgs84 srs def
xytransform = osr.CoordinateTransformation(from_srs,to_srs)            # get transform objec

In [72]:
# get UTM and lat/long coordinates for each of the sites
utm_coordinate_pairs = {}
ll_coordinate_pairs = {}

for n in range(len(lyr)):
    feature = lyr[n]
    print(type(feature))
    geom = feature.GetGeometryRef()    
    # get site geometry
    #utm_coordinate_pairs[feature['id']] = (geom.GetX(), geom.GetY()) # get x,y utm coordinates 
    #geom.Transform(xytransform)                                        # to wgs84
    #ll_coordinate_pairs[feature['id']] = (geom.GetX(), geom.GetY())  # get lon, lat



<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Feature'>
<class 'osgeo.ogr.Fe

In [59]:
# get the x and y UTM coordinates for the first site
x, y = utm_coordinate_pairs['TL_IS_2']
affine_transform = affine.Affine.from_gdal(*img.GetGeoTransform())     # affine forward transform
inverse_transform = ~affine_transform                                  # invert transform
px, py = inverse_transform * (x, y)                                    # apply to x,y coordinates
px, py = int(px + 0.5), int(py + 0.5)                                  # get new x,y as integers

# print the three coordinates (UTM, geographic, image)
print( "\n".join(["Site 1 UTM coordinates (x,y): "+"\t"*4+str((x,y)),
       " are equal to geographic coordinates (lng,lat): \t"+str(ll_coordinate_pairs['TL_IS_2']),
       " and fall within image coordinates (pixel,line):\t"+str((px,py))]) )

KeyError: 'TL_IS_2'